In [1]:
from collections import namedtuple, Counter

Hexagonal grid will be represented in [cube coordinates](https://www.redblobgames.com/grids/hexagons/#coordinates-cube).

In [2]:
HexPoint = namedtuple('HexPoint', 'x y z')

In [3]:
HexPoint.__add__ = lambda a, b: HexPoint(a.x + b.x, a.y + b.y, a.z + b.z)

In [4]:
def get_input(fname="input.txt"):
    moves = []
    with open(fname) as f:
        for line in f.readlines():
            line = line.strip()
            i = iter(line)
            cmoves = []
            while True:
                nxt = next(i, None)
                if nxt is None:
                    break
                if nxt in ('n', 's'):
                    cmoves.append(nxt + next(i))
                else:
                    cmoves.append(nxt)
            moves.append(cmoves)
    return moves

In [5]:
test_data = get_input("test.txt")

In [6]:
def convert_moves_to_coordinates(moves):
    direction_deltas = {
        'ne': HexPoint(1, 0, -1),
        'nw': HexPoint(0, 1, -1),
        'se': HexPoint(0, -1, 1),
        'sw': HexPoint(-1, 0, 1),
        'e':  HexPoint(1, -1, 0),
        'w':  HexPoint(-1, 1, 0)
    }
    origin = HexPoint(0, 0, 0)
    coordinates = []
    for move in moves:
        current = origin
        for direction in move:
            current += direction_deltas[direction]
        coordinates.append(current)
    return coordinates

In [7]:
test_coordinates = convert_moves_to_coordinates(test_data)

In [8]:
c = Counter(test_coordinates)

In [9]:
sum(v % 2 for v in c.values())

10

In [10]:
input_data = get_input("input.txt")

In [11]:
coordinates = convert_moves_to_coordinates(input_data)

In [12]:
sum(v % 2 for v in Counter(coordinates).values())

263

In [13]:
test_black_tiles = set(c for c, v in Counter(test_coordinates).items() if v % 2 == 1)

In [14]:
test_black_tiles

{HexPoint(x=-3, y=0, z=3),
 HexPoint(x=-3, y=1, z=2),
 HexPoint(x=-2, y=1, z=1),
 HexPoint(x=-2, y=2, z=0),
 HexPoint(x=-1, y=2, z=-1),
 HexPoint(x=0, y=-2, z=2),
 HexPoint(x=0, y=0, z=0),
 HexPoint(x=0, y=1, z=-1),
 HexPoint(x=2, y=-2, z=0),
 HexPoint(x=3, y=0, z=-3)}

In [15]:
def simulate(initial_black_tiles, days=100):
    direction_deltas = {
        'ne': HexPoint(1, 0, -1),
        'nw': HexPoint(0, 1, -1),
        'se': HexPoint(0, -1, 1),
        'sw': HexPoint(-1, 0, 1),
        'e':  HexPoint(1, -1, 0),
        'w':  HexPoint(-1, 1, 0)
    }
    current_black_tiles = initial_black_tiles
    for _ in range(days):
        new_black_tiles = set()
        white_tiles_to_check = {}
        for tile in current_black_tiles:
            neighbors = set(tile + delta for delta in direction_deltas.values())
            black_neighbors = neighbors & current_black_tiles
            white_neighbors = neighbors - black_neighbors
            if len(black_neighbors) in (1, 2): # remains black
                new_black_tiles.add(tile)
            for n in white_neighbors:
                if n not in white_tiles_to_check:
                    white_tiles_to_check[n] = 1
                else:
                    white_tiles_to_check[n] += 1
        for tile, black_neighbors in white_tiles_to_check.items():
            if black_neighbors == 2:
                new_black_tiles.add(tile)
        current_black_tiles = new_black_tiles
    return current_black_tiles

In [16]:
len(simulate(test_black_tiles))

2208

In [17]:
black_tiles = set(c for c, v in Counter(coordinates).items() if v % 2 == 1)

In [18]:
%%time
len(simulate(black_tiles))

CPU times: user 1.25 s, sys: 7.16 ms, total: 1.26 s
Wall time: 1.26 s


3649